<a href="https://colab.research.google.com/github/diegosfc91/EJERCICIOS-DE-CORE-Machine-Learning/blob/main/Modelos_basados_en_%C3%A1rbol_Predicci%C3%B3n_de_Precios_de_Veh%C3%ADculos_Usados_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Basados en Arbol

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np

# Cargar datos desde un archivo CSV
df = pd.read_csv('/content/Automobile_data.csv')

# Reemplazar valores faltantes representados como '?' por NaN
df.replace('?', np.nan, inplace=True)


# Separar las columnas numéricas y categóricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

## Para columnas numéricas: Reemplazar valores faltantes con la media
imputer_num = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])

## Para columnas categóricas: Reemplazar valores faltantes con el valor más frecuente
imputer_cat = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])


## Suponiendo que 'price' es la variable objetivo
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # Asegurar que 'price' sea numérica
X = df.drop(columns=['price'])
y = df['price']

## Crear el codificador y transformar las columnas categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Evitar multicolinealidad
X_categorical_encoded = pd.DataFrame(
    encoder.fit_transform(df[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols)
)

## Mantener columnas numéricas como están
X_numeric = X[numeric_cols].reset_index(drop=True)

## Combinar numéricas y categóricas transformadas
X_transformed = pd.concat([X_numeric, X_categorical_encoded], axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=43)

# Escalar las características numéricas usando StandardScaler
scaler = StandardScaler()

## Ajustar y transformar el conjunto de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

## Transformar el conjunto de prueba
X_test_scaled = scaler.transform(X_test)


# Regresión Lineal:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Paso 1: Entrenar el modelo de Regresión Lineal
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Paso 2: Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Paso 3: Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)  # Error cuadrático medio
r2 = r2_score(y_test, y_pred)            # Coeficiente de determinación

# Paso 4: Mostrar los resultados
print("Evaluación del Modelo de Regresión Lineal:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

root_mse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {root_mse:.2f}")
print("")

# Calcular el promedio de la columna 'price'
average_price = df['price'].mean()

# Mostrar el promedio
print(f"El promedio de los valores en la columna 'price' es: {average_price:.2f}")
print("")

max_price = df['price'].max()
min_price = df['price'].min()

# Mostrar los resultados
print(f"El mayor valor en la columna 'price' es: {max_price:.2f}")
print(f"El menor valor en la columna 'price' es: {min_price:.2f}")

Evaluación del Modelo de Regresión Lineal:
Mean Squared Error (MSE): 6752945.75
Coeficiente de Determinación (R²): 0.80
Root Mean Squared Error (RMSE): 2598.64

El promedio de los valores en la columna 'price' es: 13212.82

El mayor valor en la columna 'price' es: 45400.00
El menor valor en la columna 'price' es: 5118.00


# Random Forest Regressor

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# 1. Implementar y evaluar Random Forest Regressor
rf_model = RandomForestRegressor(random_state=43)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

# Calcular métricas para Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)  # Raíz del MSE
r2_rf = r2_score(y_test, y_pred_rf)

# Resultados iniciales para Random Forest
print("Random Forest Regressor:")
print(f"  MSE: {mse_rf:.2f}")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R²: {r2_rf:.2f}")


Random Forest Regressor:
  MSE: 19313572.98
  RMSE: 4394.72
  R²: 0.41


# Optimizacion

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Definir la cuadrícula de hiperparámetros para GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],        # Número de árboles en el bosque
    'max_depth': [None, 10, 20, 30],      # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],      # Muestras mínimas para dividir un nodo
    'min_samples_leaf': [1, 2, 4]         # Muestras mínimas en cada hoja
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=43),
    param_grid=param_grid,
    scoring='r2',       # Usar R² como métrica de evaluación
    cv=5,               # Validación cruzada con 5 particiones
    verbose=1,          # Mostrar progreso
    n_jobs=-1           # Usar todos los núcleos disponibles
)

# Entrenar GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Mostrar los mejores hiperparámetros
print("\nMejores Hiperparámetros:")
print(grid_search.best_params_)

# Evaluar el modelo optimizado
best_rf_model = grid_search.best_estimator_
y_pred_optimized = best_rf_model.predict(X_test_scaled)

# Calcular métricas de evaluación
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
rmse_optimized = np.sqrt(mse_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

# Mostrar resultados del modelo optimizado
print("\nEvaluación del Modelo Optimizado:")
print(f"  MSE: {mse_optimized:.2f}")
print(f"  RMSE: {rmse_optimized:.2f}")
print(f"  R²: {r2_optimized:.2f}")


Fitting 5 folds for each of 108 candidates, totalling 540 fits

Mejores Hiperparámetros:
{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

Evaluación del Modelo Optimizado:
  MSE: 19842963.11
  RMSE: 4454.54
  R²: 0.40
